# PSからPLを制御するためのプログラム

In [78]:
N = 10              # 最大文字数

# kmj_data = ['(　́ω`)ノ']
kmj_data = ['(*　̄∇　̄)ノ']

## 前処理

### 使用できる文字のリストを読み込み

In [2]:
char_list = []
with open('char_list.txt' , 'r', encoding='utf-8') as file:
  for line in file:
    char_list.append(line.replace('\n', ''))

### 文字を整数値に変換

In [79]:
# 文字リストの番号を顔文字中の番号を対応付け
kmj_index = []
for kmj in kmj_data:
  kmj = list(kmj)
  kmj += ['<PAD>' for _ in range(N - len(kmj))]
  temp = []
  for c in kmj:
    try:
      temp.append(char_list.index(c))
    except:
      temp.append(char_list.index('<UNK>'))
  kmj_index.append(temp)

In [80]:
kmj_index

[[2, 18, 7, 41, 108, 7, 41, 5, 33, 0]]

## FPGAとの通信

In [5]:
from pynq import PL
PL.reset()

### FPGA書き込み

In [6]:
from pynq import Overlay

ol = Overlay("kmj_gen_v1_0.bit")

### IPコア定義

In [7]:
dma = ol.axi_dma_0
dma_send = dma.sendchannel
dma_recv = dma.recvchannel

In [8]:
from pynq import MMIO
top_mmio = MMIO(ol.ip_dict['top_0']['phys_addr'], ol.ip_dict['top_0']['addr_range'])

### AXI Stream でデータを書き込み

In [9]:
from pynq import allocate
import numpy as np

data_size = N
input_buffer = allocate(shape=(data_size,), dtype=np.uint8)
output_buffer = allocate(shape=(data_size,), dtype=np.uint8)

In [81]:
for i in range(data_size):
    input_buffer[i] = kmj_index[0][i]

In [82]:
input_buffer, output_buffer

(PynqBuffer([  2,  18,   7,  41, 108,   7,  41,   5,  33,   0], dtype=uint8),
 PynqBuffer([17,  2, 18,  7, 61, 26,  7, 18,  5, 33], dtype=uint8))

In [83]:
# データ送信
dma_send.transfer(input_buffer)

### AXI LITE で回路全体を制御

In [84]:
top_mmio.write(0, 0)

In [85]:
# slv_reg1[1:0] : mode
top_mmio.write(4, 0b00)

In [86]:
# slv_reg0[2:0] : {set, run, rst_n}
top_mmio.write(0, 3)

In [87]:
# slv_reg2[0] : finish
top_mmio.read(8)

1

### AXI Stream でデータを読み込み

In [88]:
dma_recv.transfer(output_buffer)

In [89]:
output_buffer

PynqBuffer([ 2, 18,  7, 26, 61,  7, 26,  5, 33,  0], dtype=uint8)

## 整数列を文字列に変換

In [90]:
def convert_str(x):
  x = np.array(char_list)[x]
  x = [c for c in x if c not in ['<PAD>', '<UNK>']]

  return ''.join(x)

In [91]:
print(kmj_data[0])
print(convert_str(output_buffer))

(*　̄∇　̄)ノ
(*　́Д　́)ノ
